In [5]:
import pygrib
import os
import csv
import numpy as np
import math
from csv import reader

with open('../lats.csv', 'r') as csv_file:
    csv_reader = reader(csv_file)
    lats = list(csv_reader)
with open('../lons.csv', 'r') as csv_file:
    csv_reader = reader(csv_file)
    lons = list(csv_reader)

for i in range(79):
    for j in range(79):
        lats[i][j] = float(lats[i][j])
        lons[i][j] = float(lons[i][j])

IDX = [79*67 + 13, 79*53 + 32, 79*42 + 70]
MODEL = ['HPA_KF', 'HFA_KF_c0', 'HPA_YS', 'HFA_YS_c0']

for i in range(len(MODEL)):
    model = MODEL[i]
    with open('../Ex_csv/' + model + '_ex.csv', 'r') as csv_file:
        csv_reader = reader(csv_file)
        STM = list(csv_reader)
    with open('../Ex_csv/' + model + '_ex_idx.csv', 'r') as csv_file:
        csv_reader = reader(csv_file)
        INDEX = list(csv_reader)
    for j in range(len(INDEX[0])):
        mx = 0
        for k in range(79 * 79):
            mx = max(mx, float(STM[k][j]))
        for k in range(79 * 79):
            STM[k][j] = float(STM[k][j]) / mx
    
    for j in range(len(IDX)):
        idx = IDX[j]
        print(model + 'の' + str(idx) + 'のExposureの平均値：', np.mean(STM[idx]))
    
    with open('../Ex_csv/' + model + '_ex_ratio.csv', 'w') as file:
        writer = csv.writer(file, lineterminator='\n')
        writer.writerows(STM)


HPA_KFの5306のExposureの平均値： 0.20814588838477496
HPA_KFの4219のExposureの平均値： 0.22016285079164136
HPA_KFの3388のExposureの平均値： 0.20030740036641312
HFA_KF_c0の5306のExposureの平均値： 0.18056684907022252
HFA_KF_c0の4219のExposureの平均値： 0.19705273872601747
HFA_KF_c0の3388のExposureの平均値： 0.1996405347707802
HPA_YSの5306のExposureの平均値： 0.20595618054743575
HPA_YSの4219のExposureの平均値： 0.2152504415135004
HPA_YSの3388のExposureの平均値： 0.20931736115494132
HFA_YS_c0の5306のExposureの平均値： 0.19515630316561686
HFA_YS_c0の4219のExposureの平均値： 0.20012865454304596
HFA_YS_c0の3388のExposureの平均値： 0.20223037418138165
